# MultiQueryRetriever

거리 기반 벡터 데이터베이스 검색은 고차원 공간에서의 쿼리 임베딩(표현)과 '거리'를 기준으로 유사한 임베딩을 가진 문서를 찾는 방식입니다. 하지만 쿼리의 세부적인 차이나 임베딩이 데이터의 의미를 제대로 포착하지 못할 경우, 검색 결과가 달라질 수 있습니다. 이를 수동으로 조정하는 프롬프트 엔지니어링이나 튜닝 작업은 번거로울 수 있습니다.

이런 문제를 해결하기 위해, `MultiQueryRetriever` 는 주어진 사용자 입력 쿼리에 대해 다양한 관점에서 여러 쿼리를 자동으로 생성하는 LLM(Language Learning Model)을 활용해 프롬프트 튜닝 과정을 자동화합니다.

이 방식은 각각의 쿼리에 대해 관련 문서 집합을 검색하고, 모든 쿼리를 아우르는 고유한 문서들의 합집합을 추출해, 잠재적으로 관련된 더 큰 문서 집합을 얻을 수 있게 해줍니다. 여러 관점에서 동일한 질문을 생성함으로써, `MultiQueryRetriever` 는 거리 기반 검색의 제한을 일정 부분 극복하고, 더욱 풍부한 검색 결과를 제공할 수 있습니다.


- `WebBaseLoader`를 사용하여 블로그 게시물(https://lilianweng.github.io/posts/2023-06-23-agent/)을 로드합니다.
- `RecursiveCharacterTextSplitter`를 사용하여 로드된 데이터를 청크 크기 500자와 청크 간 중복 0으로 분할합니다.
- `OpenAIEmbeddings`를 사용하여 임베딩을 생성합니다.
- 분할된 문서와 생성된 임베딩을 사용하여 `Chroma` 벡터 데이터베이스를 구축합니다.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [116]:
# 샘플 벡터DB 구축
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 블로그 포스트 로드
loader = WebBaseLoader(
    "https://teddylee777.github.io/openai/openai-assistant-tutorial/", encoding="utf-8"
)

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

# 임베딩 정의
openai_embedding = OpenAIEmbeddings()

# 벡터DB 생성
db = FAISS.from_documents(docs, openai_embedding)

# retriever 생성
retriever = db.as_retriever()

# 문서 검색
query = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
relevant_docs = retriever.get_relevant_documents(query)

# 검색된 문서의 개수 출력
len(relevant_docs)

4

검색된 결과 중 1개 문서의 내용을 출력합니다.


In [118]:
# 1번 문서를 출력합니다.
print(relevant_docs[1].page_content)

가장 강력한 도구로서, Assistant에게 사용자 정의 함수를 지정할 수 있습니다. 이는 Chat Completions API에서의 함수 호출과 매우 유사합니다.


Function calling(함수 호출) 도구를 사용하면 Assistant 에게 사용자 정의 함수 를 설명하여 호출해야 하는 함수를 인자와 함께 지능적으로 반환하도록 할 수 있습니다.


Assistant API는 실행 중에 함수를 호출할 때 실행을 일시 중지하며, 함수 호출 결과를 다시 제공하여 Run 실행을 계속할 수 있습니다. (이는 사용자 피드백을 받아 재게할 수 있는 의미이기도 합니다. 아래 튜토리얼에서 상세히 다룹니다).


## 기본 예제

`MultiQueryRetriever` 에 사용할 LLM을 지정하고 질의 생성에 사용하면, retriever가 나머지 작업을 처리합니다.


- `ChatOpenAI`를 사용하여 LLM을 초기화합니다.
- `MultiQueryRetriever.from_llm` 메서드를 사용하여 `retriever_from_llm` 객체를 생성합니다.
  - 이 메서드는 벡터 저장소의 검색기(`vectordb.as_retriever()`)와 LLM(`llm`)을 인자로 받습니다.

`MultiQueryRetriever`는 LLM을 활용하여 질문을 분석하고, 관련된 문서를 벡터 저장소에서 검색하여 최종 답변을 생성하는 역할을 합니다. 이를 통해 질문에 대한 보다 정확하고 포괄적인 답변을 제공할 수 있습니다.


In [124]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI


# ChatOpenAI 언어 모델을 초기화합니다. temperature는 0으로 설정합니다.
llm = ChatOpenAI(temperature=0)

multiquery_retriever = MultiQueryRetriever.from_llm(  # MultiQueryRetriever를 언어 모델을 사용하여 초기화합니다.
    # 벡터 데이터베이스의 retriever와 언어 모델을 전달합니다.
    retriever=db.as_retriever(),
    llm=llm,
)

아래는 다중 쿼리를 생성하는 중간 과정을 디버깅하기 위하여 실행하는 코드입니다.

- `logging.getLogger("langchain.retrievers.multi_query")` 함수를 사용하여 `"langchain.retrievers.multi_query"` 로거를 가져옵니다.
- `setLevel(logging.INFO)` 메서드를 호출하여 해당 로거의 로그 레벨을 `INFO`로 설정합니다. 이는 `INFO` 레벨 이상의 로그 메시지만 출력되도록 합니다.


In [120]:
# 쿼리에 대한 로깅 설정
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

- `retriever_from_llm` 객체의 `get_relevant_documents` 메서드를 호출하여 `question`과 관련된 문서를 검색합니다.
- `unique_docs`에는 검색된 관련 문서가 저장됩니다.
- `len(unique_docs)`를 통해 검색된 관련 문서의 개수를 확인합니다.


In [128]:
# 질문을 정의합니다.
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."
# 문서 검색
relevant_docs = multiquery_retriever.get_relevant_documents(query=question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. OpenAI Assistant API의 Functions는 어떻게 사용되는지 설명해 주실 수 있나요?', '2. OpenAI Assistant API의 Functions를 어떻게 활용할 수 있는지 알려주세요.', '3. OpenAI Assistant API의 Functions를 어떻게 구현하고 활용할 수 있는지에 대해 자세히 설명해 주세요.']


검색된 문서 개수: 4
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용


## LCEL Chain 활용하는 방법

- 사용자 정의 프롬프트 정의하고, 정의한 프롬프트와 함께 Chain 을 생성합니다.
- Chain 은 사용자의 질문을 입력 받으면 (아래의 예제에서는) 5개의 질문을 생성한 뒤 `"\n"` 구분자로 구분하여 생성된 5개 질문을 반환합니다.


In [130]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.(5개의 질문을 생성하도록 프롬프트를 작성하였습니다)
prompt = PromptTemplate.from_template(
    """You are an AI language model assistant. 
Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. 
By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search. 
Your response should be a list of values separated by new lines, eg: `foo\nbar\nbaz\n`

#ORIGINAL QUESTION: 
{question}
"""
)

# 언어 모델 인스턴스를 생성합니다.
llm = ChatOpenAI(temperature=0)

# LLMChain을 생성합니다.
chain = {"question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

# 질문을 정의합니다.
question = "OpenAI Assistant API의 Functions 사용법에 대해 알려주세요."

# 체인을 실행하여 생성된 다중 쿼리를 확인합니다.
multi_queries = chain.invoke({"question": question})
# 결과를 확인합니다.(5개 질문 생성)
multi_queries

"1. How can I learn about the usage of Functions in the OpenAI Assistant API?\n2. Can you provide information on how to use Functions in the OpenAI Assistant API?\n3. What is the proper way to utilize Functions in the OpenAI Assistant API?\n4. Please explain the usage guidelines for Functions in the OpenAI Assistant API.\n5. I'm looking for instructions on how to implement Functions in the OpenAI Assistant API."

이전에 생성한 Chain을 `MultiQueryRetriever` 에 전달하여 retrieve 할 수 있습니다.


In [131]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    llm=chain, retriever=db.as_retriever()
)

`MultiQueryRetriever`를 사용하여 문서를 검색하고 결과를 확인합니다.


In [134]:
# 결과
relevant_docs = multiquery_retriever.get_relevant_documents(query=question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['How can I use the Functions of the OpenAI Assistant API?', 'Can you provide guidance on using the Functions of the OpenAI Assistant API?', 'What are the steps to utilize the Functions of the OpenAI Assistant API?', 'I need help understanding how to use the Functions of the OpenAI Assistant API.', 'Please explain the usage of the Functions in the OpenAI Assistant API.']


검색된 문서 개수: 5
OpenAI의 새로운 Assistants API는 대화와 더불어 강력한 도구 접근성을 제공합니다. 본 튜토리얼은 OpenAI Assistants API를 활용하는 내용을 다룹니다. 특히, Assistant API 가 제공하는 도구인 Code Interpreter, Retrieval, Functions 를 활용하는 방법에 대해 다룹니다. 이와 더불어 파일을 업로드 하는 내용과 사용자의 피드백을 제출하는 내용도 튜토리얼 말미에 포함하고 있습니다.



주요내용
